In [ ]:
!pwd

/content


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dverefym
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dverefym
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=5e10033b38f805781b82d1f93e354775da7d1cdc3d6c6f42a9071e2a59dd8681
  Stored in directory: /tmp/pip-ephem-wheel-cache-6pl79w8t/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name prg1.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>
#define TILE_WIDTH 2
#define WIDTH 4
#define MASK_WIDTH 3

__global__ void convolution(int* input, int* mask, int* output) {
    int row = threadIdx.y + blockDim.y * blockIdx.y;
    int col = threadIdx.x + blockDim.x * blockIdx.x;
    int sum = 0;
    for (int i = 0; i < MASK_WIDTH; i++) {
        for (int j = 0; j < MASK_WIDTH; j++) {
            int inputRow = row + i - MASK_WIDTH / 2;
            int inputCol = col + j - MASK_WIDTH / 2;
            if (inputRow >= 0 && inputRow < WIDTH && inputCol >= 0 && inputCol < WIDTH) {
                sum += input[inputRow * WIDTH + inputCol] * mask[i * MASK_WIDTH + j];
            }
        }
    }
    output[row * WIDTH + col] = sum;
}

int main() {
    int *input, *mask, *output, *d_input, *d_mask, *d_output;
    input = (int*)malloc(sizeof(int) * WIDTH * WIDTH);
    output = (int*)malloc(sizeof(int) * WIDTH * WIDTH);
    mask = (int*)malloc(sizeof(int) * MASK_WIDTH * MASK_WIDTH);
    printf("Enter the (4x4) input matrix:\n");
    for (int i = 0; i < WIDTH; i++) {
        for (int j = 0; j < WIDTH; j++) {
            scanf("%d", &input[i * WIDTH + j]);
        }
    }
    printf("Enter the (3x3) mask matrix:\n");
    for (int i = 0; i < MASK_WIDTH; i++) {
        for (int j = 0; j < MASK_WIDTH; j++) {
            scanf("%d", &mask[i * MASK_WIDTH + j]);
        }
    }
    cudaMalloc((void**)&d_input, WIDTH * WIDTH * sizeof(int));
    cudaMalloc((void**)&d_mask, MASK_WIDTH * MASK_WIDTH * sizeof(int));
    cudaMalloc((void**)&d_output, WIDTH * WIDTH * sizeof(int));
    cudaMemcpy(d_input, input, WIDTH * WIDTH * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, mask, MASK_WIDTH * MASK_WIDTH * sizeof(int), cudaMemcpyHostToDevice);
    dim3 grid_conf(WIDTH / TILE_WIDTH, WIDTH / TILE_WIDTH);
    dim3 block_conf(TILE_WIDTH, TILE_WIDTH);
    convolution<<<grid_conf, block_conf>>>(d_input, d_mask, d_output);
    cudaMemcpy(output, d_output, WIDTH * WIDTH * sizeof(int), cudaMemcpyDeviceToHost);
    printf("After performing convolution:\n");
    for (int i = 0; i < WIDTH; i++) {
        for (int j = 0; j < WIDTH; j++) {
            printf("%d ", output[i * WIDTH + j]);
        }
        printf("\n");
    }
    cudaFree(d_input);
    cudaFree(d_mask);
    cudaFree(d_output);
    free(input);
    free(output);
    free(mask);
}


'File written in /content/src/prg1.cu'

In [ ]:
!nvcc /content/src/prg1.cu -o /content/src/prg1

In [ ]:
!/content/src/prg1

Enter the (4x4) input matrix:
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 16
Enter the (3x3) mask matrix:
1 2 3
4 5 6
7 8 9
After performing convolution:
111 178 217 145 
231 348 393 252 
363 528 573 360 
197 274 295 175 


In [ ]:
%%cuda --name prg2.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>
#define BLOCK_WIDTH 2
#define TILE_WIDTH 2
#define WIDTH 4

__global__ void matMul(int* a,int* b,int* c){
    __shared__ int MD[TILE_WIDTH][TILE_WIDTH];
    __shared__  int ND[TILE_WIDTH][TILE_WIDTH];
    int m;
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by*TILE_WIDTH + ty;
    int col = bx*TILE_WIDTH + tx;
    int pval = 0;
    for(m = 0;m < WIDTH/TILE_WIDTH;m++){
        MD[tx][ty] = a[row*WIDTH + m * TILE_WIDTH+tx];
        ND[tx][ty] = b[(m*TILE_WIDTH + ty)* WIDTH+col];
        __syncthreads();
        for(int k = 0;k < TILE_WIDTH;k++){
            pval += MD[ty][k]*ND[k][tx];
        }

        __syncthreads();
    }
    c[row*WIDTH + col] = pval;
}


int main(){
    int* matA,*matB,*matC,*d_a,*d_b,*d_c;
    matA = (int*)malloc(WIDTH*WIDTH*sizeof(int));
    printf("Enter the elements of 4x4 matA:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
      scanf("%d",&matA[i]);
    matB = (int*)malloc(WIDTH*WIDTH*sizeof(int));
    printf("Enter the elements of 4x4 matB:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
      scanf("%d",&matB[i]);
    matC = (int*)malloc(WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&d_a,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&d_b,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&d_c,sizeof(int)*WIDTH*WIDTH);
    cudaMemcpy(d_a,matA,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,matB,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    dim3 grid_conf(WIDTH/TILE_WIDTH,WIDTH/TILE_WIDTH);
    dim3 block_conf(TILE_WIDTH,TILE_WIDTH);
    matMul<<<grid_conf,block_conf>>>(d_a,d_b,d_c);
    cudaMemcpy(matC,d_c,sizeof(int)*WIDTH*WIDTH,cudaMemcpyDeviceToHost);
    printf("Result:\n");
    for(int i = 0;i < WIDTH;i++){
      for(int j = 0;j < WIDTH;j++){
        printf("%d ",matC[i*WIDTH + j]);
      }
      printf("\n");
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(matA);
    free(matB);
    free(matC);
    return 0;
}

'File written in /content/src/prg2.cu'

In [ ]:
!nvcc /content/src/prg2.cu -o /content/src/prg2

In [ ]:
!/content/src/prg2

Enter the elements of 4x4 matA:
1 2 3 4
5 6 7 8
9 10 11 12
13 14 15 16
Enter the elements of 4x4 matB:
1 2 3 4 
5 6 7 8
9 10 11 12 
13 14 15 16
Result:
108 172 140 204 
130 210 170 250 
284 476 380 572 
306 514 410 618 


In [ ]:
%%cuda --name prg3.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void odd(int *a, int n) {
    int tid = blockDim.x*blockIdx.x + threadIdx.x ;
    if (tid%2 != 0 && tid + 1 < n) {
        if (a[tid] > a[tid + 1]) {
            int temp = a[tid];
            a[tid] = a[tid + 1];
            a[tid + 1] = temp;
        }
    }
}

__global__ void even(int *a, int n) {
    int tid = blockDim.x*blockIdx.x + threadIdx.x;
    if (tid % 2 == 0 && tid + 1 < n) {
        if (a[tid] > a[tid + 1]) {
            int temp = a[tid];
            a[tid] = a[tid + 1];
            a[tid + 1] = temp;
        }
    }
}

int main() {
    int *a, n, *d_a;
    printf("Enter the size of the array:\n");
    scanf("%d", &n);
    printf("Enter the array elements:\n");
    a = (int*)malloc(sizeof(int) * n);
    for (int i = 0; i < n; i++) {
        scanf("%d", &a[i]);
    }
    cudaMalloc((void**)&d_a, sizeof(int) * n);
    cudaMemcpy(d_a, a, sizeof(int) * n, cudaMemcpyHostToDevice);
    for (int i = 0; i < n / 2; i++) {
        odd<<<1, n>>>(d_a, n);
        even<<<1, n>>>(d_a, n);
    }
    cudaMemcpy(a, d_a, sizeof(int) * n, cudaMemcpyDeviceToHost);
    printf("Result:\n");
    for (int i = 0; i < n; i++)
        printf("%d ", a[i]);
    cudaFree(d_a);
    free(a);
    return 0;
}


'File written in /content/src/prg3.cu'

In [ ]:
!nvcc /content/src/prg3.cu -o /content/src/prg3

In [ ]:
!/content/src/prg3

Enter the size of the array:
4
Enter the array elements:
9 1 0 11
Result:
0 1 9 11 

In [ ]:
%%cuda --name prg4.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#define MASK_SIZE 3

__constant__ int mask[MASK_SIZE];

__global__ void convolution(int* input, int* output, int width) {
    int tid = threadIdx.x + blockDim.x * blockIdx.x;
    if (tid < width) {
        int res = 0;
        for (int i = 0; i < MASK_SIZE; i++) {
            int idx = tid - MASK_SIZE / 2 + i;
            if (idx >= 0 && idx < width) {
                res += input[idx] * mask[i];
            }
        }
        output[tid] = res;
    }
}

int main() {
    int width, *input, *output, *d_input, *d_output;
    printf("Enter the width:\n");
    scanf("%d", &width);
    input = (int*)malloc(sizeof(int) * width);
    output = (int*)malloc(sizeof(int) * width);

    printf("Enter the array elements:\n");
    for (int i = 0; i < width; i++)
        scanf("%d", &input[i]);

    printf("Enter the mask elements:\n");
    int maskElements[MASK_SIZE];
    for (int i = 0; i < MASK_SIZE; i++)
        scanf("%d", &maskElements[i]);

    // Copy the mask to constant memory
    cudaMemcpyToSymbol(mask, maskElements, MASK_SIZE * sizeof(int));

    cudaMalloc((void**)&d_input, sizeof(int) * width);
    cudaMalloc((void**)&d_output, sizeof(int) * width);
    cudaMemcpy(d_input, input, sizeof(int) * width, cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (width + blockSize - 1) / blockSize;
    convolution<<<gridSize, blockSize>>>(d_input, d_output, width);
    cudaMemcpy(output, d_output, sizeof(int) * width, cudaMemcpyDeviceToHost);

    printf("Result:\n");
    for (int i = 0; i < width; i++) {
        printf("%d ", output[i]);
    }

    cudaFree(d_input);
    cudaFree(d_output);
    free(input);
    free(output);

    return 0;
}


'File written in /content/src/prg4.cu'

In [ ]:
!nvcc /content/src/prg4.cu -o /content/src/prg4

In [ ]:
!/content/src/prg4

Enter the width:
5
Enter the array elements:
1 2 3 4 5
Enter the mask elements:
1 2 3 4
Result:
8 14 20 26 14 

In [ ]:
%%cuda --name prg1.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>
#define BLOCK_WIDTH 2
#define TILE_WIDTH 2
#define WIDTH 4

__global__ void matMul(int* a,int* b,int* c){
    __shared__ int MD[TILE_WIDTH][TILE_WIDTH];
    __shared__ int ND[TILE_WIDTH][TILE_WIDTH];
    int m;
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by*TILE_WIDTH + ty;
    int col = bx*TILE_WIDTH + tx;
    int pval = 0;
    for(m=0;m < TILE_WIDTH/WIDTH;m++){
        MD[tx][ty] = a[row*WIDTH + m*TILE_WIDTH + tx];
        ND[tx][ty] = b[m*TILE_WIDTH + ty*WIDTH + col];
        __syncthreads();
        for(int k = 0;k < TILE_WIDTH;k++){
            pval += MD[ty][k]*ND[k][tx];
        }
        __syncthreads();
    }
    c[row*WIDTH + col] = pval;
}


int main(){
    int* a,*b,*c,*da,*db,*dc;
    a = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    b = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    c = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    printf("Enter a 4x4 matrix A:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
        scanf("%d",&a[i]);
    printf("Enter a 4x4 matrix B:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
        scanf("%d",&b[i]);
    cudaMalloc((void**)&da,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&db,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&dc,sizeof(int)*WIDTH*WIDTH);
    cudaMemcpy(da,a,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    cudaMemcpy(db,b,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    dim3 grid_conf(WIDTH/TILE_WIDTH,WIDTH/TILE_WIDTH);
    dim3 block_conf(TILE_WIDTH,TILE_WIDTH);
    matMul<<<grid_conf,block_conf>>>(da,db,dc);
    cudaMemcpy(c,dc,sizeof(int)*WIDTH*WIDTH,cudaMemcpyDeviceToHost);
    printf("Result:\n");
    for(int i = 0;i < WIDTH;i++){
        for(int j = 0;j < WIDTH;j++){
            printf("%d ", c[i*WIDTH + j]);
        }
        printf("\n");
    }
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
    free(a);
    free(b);
    free(c);
    return 0;
}


'File written in /content/src/prg1.cu'

In [ ]:
!nvcc /content/src/prg1.cu -o /content/src/prg1

In [ ]:
!/content/src/prg1

Enter a 4x4 matrix A:
1 2 3 4
5 6 7 8 
9 10 11 12
13 14 15 16
Enter a 4x4 matrix B:
1 2 3 4
5 6 7 8 
9 10 11 12
13 14 15 16
Result:
0 0 0 0 
0 0 0 0 
0 0 0 0 
0 0 0 0 


In [5]:
%%cuda --name prg1.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>
#define WIDTH 4
#define TILE_WIDTH 4
__global__ void matMul(int* a,int* b,int* c){
    __shared__ int MD[TILE_WIDTH][TILE_WIDTH];
    __shared__ int ND[TILE_WIDTH][TILE_WIDTH];
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int res = 0;
    int row = by*TILE_WIDTH + ty;
    int col = bx*TILE_WIDTH + tx;
    int m;
    for(m = 0; m < WIDTH/TILE_WIDTH;m++){
        MD[tx][ty] = a[row*WIDTH + m*TILE_WIDTH+ tx];
        ND[tx][ty] = a[m*TILE_WIDTH + ty*WIDTH+ col];
        __syncthreads();
        for(int k = 0;k < TILE_WIDTH;k++){
            res += MD[ty][k]*ND[k][tx];
        }
        __syncthreads();
    }
    c[row*WIDTH + col] = res;
}

int main(){
    int* a,*b,*c,*da,*db,*dc;
    a = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    b = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    c = (int*)malloc(sizeof(int)*WIDTH*WIDTH);
    printf("Enter the matrix A:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
      scanf("%d",&a[i]);
    printf("Enter the matrix B:\n");
    for(int i = 0;i < WIDTH*WIDTH;i++)
      scanf("%d",&b[i]);
    cudaMalloc((void**)&da,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&db,sizeof(int)*WIDTH*WIDTH);
    cudaMalloc((void**)&dc,sizeof(int)*WIDTH*WIDTH);
    cudaMemcpy(da,a,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    cudaMemcpy(db,b,sizeof(int)*WIDTH*WIDTH,cudaMemcpyHostToDevice);
    dim3 grid_conf(WIDTH/TILE_WIDTH,WIDTH/TILE_WIDTH);
    dim3 block_conf(TILE_WIDTH,TILE_WIDTH);
    matMul<<<grid_conf,block_conf>>>(da,db,dc);
    cudaMemcpy(c,dc,sizeof(int)*WIDTH*WIDTH,cudaMemcpyDeviceToHost);
    printf("Result:\n");
    for(int i = 0;i < WIDTH;i++){
        for(int j = 0; j < WIDTH;j++){
            printf("%d ", c[i*WIDTH + j]);
        }
        printf("\n");
    }
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
    return 0;
}


'File written in /content/src/prg1.cu'

In [6]:
!nvcc /content/src/prg1.cu -o /content/src/prg1

In [7]:
!/content/src/prg1

Enter the matrix A:
1 2 3 4
5 6 7 8 
9 10 11 12 
13 14 15 16
Enter the matrix B:
1 2 3 4
5 6 7 8 
9 10 11 12
13 14 15 16
Result:
90 202 314 426 
100 228 356 484 
110 254 398 542 
120 280 440 600 


In [8]:
%%cuda --name prg2.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>

__global__ void odd(int* a,int n){
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    if(tid % 2 != 0 && tid + 1 < n){
        if(a[tid] > a[tid + 1]){
            int temp = a[tid];
            a[tid] = a[tid + 1];
            a[tid + 1] = temp;
        }
    }
}

__global__ void even(int* a,int n){
    int tid = blockIdx.x*blockDim.x + threadIdx.x;
    if(tid % 2 == 0 && tid + 1 < n){
        if(a[tid] > a[tid + 1]){
            int temp = a[tid];
            a[tid] = a[tid + 1];
            a[tid + 1] = temp;
        }
    }
}

int main(){
    int* a,n,*da;
    printf("Enter n:\n");
    scanf("%d", &n);
    printf("Enter the array elements:\n");
    a = (int*)malloc(sizeof(int)*n);
    for(int i = 0;i < n;i++){
        scanf("%d",&a[i]);
    }
    cudaMalloc((void**)&da,sizeof(int)*n);
    cudaMemcpy(da,a,sizeof(int)*n,cudaMemcpyHostToDevice);
    for(int i = 0;i < n;i++){
        odd<<<1,n>>>(da,n);
        even<<<1,n>>>(da,n);
    }
    cudaMemcpy(a,da,sizeof(int)*n,cudaMemcpyDeviceToHost);
    printf("Result:\n");
    for(int i = 0;i < n;i++){
        printf("%d ",a[i]);
    }
    cudaFree(da);
    free(a);
    return 0;
}

'File written in /content/src/prg2.cu'

In [9]:
!nvcc /content/src/prg2.cu -o /content/src/prg2

In [10]:
!/content/src/prg2

Enter n:
6
Enter the array elements:
10 1 9 0 7 4
Result:
0 1 4 7 9 10 

In [ ]:
%%cuda --name prg3.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda_runtime.h>
#define TILE_SIZE 32

__global__ void convolution(int* N,int* M,int* P,int width,int mask_width){
    __shared__  int N_tile[TILE_SIZE + 2 * (MASK_WIDTH / 2)];
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int start = i - (mask_width/2);
    int local_index = threadIdx.x  + mask_width/2;
    if(start >= 0 && start < width){
        N_tile[local_index] = start;
    } else{
        N_tile[local_index] = 0;
    }
    __syncthreads();
    int pval = 0;
    for(int j = 0;j < mask_width;j++){
        pval += N_tile[local_index + j ]
    }

}